<a href="https://colab.research.google.com/github/dxdiego5/MachineLearning/blob/main/KNN_IRIS_CLASSIF_ACURRACIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [247]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import numpy as np
from sklearn.metrics import accuracy_score


def distance(element,x):

	dist = 0
	for i in range(len(element)):
		dist += (element[i]-x[i])**2
	return dist**(0.5)


def most_close_element(element,x_train,y_train,y_verification):

	min_distance = float('inf')
	class_most_close = 0
	idx_most_close = 0


	for i in range(np.shape(x_train)[0]):
		current_distance = distance(element,x_train[i])
		if current_distance < min_distance and y_verification[i]:
			min_distance = current_distance
			class_most_close = y_train[i]
			idx_most_close = i

	y_verification[idx_most_close] = False
	return class_most_close,y_verification


def most_common(most_classes_close):
	return max(set(most_classes_close),key=most_classes_close.count)


def filtro_quartil_IIQ(df, col_):
    
  Q1 = np.percentile(df[col_], 25)
  Q3 = np.percentile(df[col_], 75)

  C = 1.5

  IIQ = Q3 - Q1
  LI = Q1 - (C * IIQ)
  LS = Q3 + (C * IIQ)
  
  df_mask= df[col_] > LI
  amplit_pos = df[df_mask]
  
  df_mask= amplit_pos[col_] < LS
  df = amplit_pos[df_mask]
  # dataframe atualizado sem outliers
  return df

def main():

	data = pd.read_csv('iris.data')
	# data = pd.read_csv('iris_modificado.csv', sep=';')
	data.columns = ['Comprimento_Sepal', 'Largura_Sepal', 'Comprimento_Petal', 'Largura_Petal', 'Classe']


	################
	## Se descomentar o codigo abaixo ele ira separar as classes em 
	## dataframe disitintos e analisar com filtro de quartil 
	## cada um dele e remover possiveis outliers
	################


	##### INICIO #####
	
	# data.dropna(inplace=True)
	# columns = ['Comprimento_Sepal', 'Largura_Sepal', 'Comprimento_Petal', 'Largura_Petal']
	# data[columns] = data[columns].apply(lambda x: x.str.replace(',', '.').astype('float'))

	# classes = data.Classe.unique()
	# df_classe = {}

	# for class_ in classes:
	# 	df_classe[class_] = data.query(f"Classe == '{class_}'")

	# # Iris-setosa
	# # Iris-versicolor
	# # Iris-virginica

	# # Filtragem com quartil retirando outliers
	# for class_ in classes:
	# 	for col_ in data.columns[:-1]:
	# 		df_classe[class_] = filtro_quartil_IIQ(df_classe[class_], col_)

	# # Concatenação dos daframes separados da analise 
	# data = pd.concat([df_classe[class_]  for class_ in classes]).reset_index(drop=True)
	
	##### FIM #####


	# Modelo
	x_train,x_test =  train_test_split(data, test_size=0.3, shuffle=True)

	# Encodner label classes
	# label_encoder = LabelEncoder()
	# data['Classe'] = label_encoder.fit_transform(data['Classe'])

	y_train = list(x_train['Classe'])
	del x_train['Classe']

	y_test = list(x_test['Classe'])
	del x_test['Classe']

	x_train = normalize(x_train, axis=1)
	x_test = normalize(x_test, axis=1)
	k = 3
	hits = 0
	for i in range(np.shape(x_test)[0]):

		most_classes_close = [0]*k
		y_verification = [True]*len(y_train)
		for j in range(k):
			most_classes_close[j], y_verification = most_close_element(x_test[i], x_train, y_train, y_verification)
		
		if most_common(most_classes_close) == y_test[i]:
			hits += 1


	print(f'Total de : {np.shape(x_test)[0]}, Acertos: {hits}, Acurácia de: {round((hits/np.shape(y_test)[0])* 100, 2)}%')


In [248]:
main()

Total de : 45, Acertos: 42, Acurácia de: 93.33%
